In [9]:
# Following code is copied from the documentation for the dataset being used
# Dataset and documentation: https://dev.socrata.com/foundry/data.lacity.org/9yda-i4ya
# Import required libraries
import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.lacity.org", None)

# All results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("9yda-i4ya", limit = 60000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
results_df.head()

,building_address,building_id,compliance,postal_code,program_year,apn,building_programclass,total_ghg_emissions,percent_difference_2,percent_difference_1,...,number_of_buildings,occupancy,outdoor_water_use,primary_property_1,site_eui,source_eui,water_use,weather_normalized_3,weather_normalized_4,year_built
0,14558 KESWICK ST,425463899040,NOT COMPLIED,91405,2021,028,"20,000 to 49,999 sqft",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5500 S FIGUEROA ST,476102819647,NOT COMPLIED,90037,2020,054,"20,000 to 49,999 sqft",138.1,29.5,29.5,...,1,100,Not Available,Multifamily Housing,54.7,99.9,2859.8,55.3,100.5,2001
2,11340 W OLYMPIC BLVD,427804836294,COMPLIED,90064,2020,017,"50,000 to 99,999 sqft",170.6,-47.8,-47.8,...,1,75,Not Available,Office,30.3,84.9,1052.3,Not Available,Not Available,1975
3,5650 FAIR AVE,448460885789,COMPLIED,91601,2021,011,"20,000 to 49,999 sqft",65.9,-27,-27,...,1,100,Not Available,Multifamily Housing,38.4,59.8,1.6,39.1,60.6,2010
4,4715 S ALAMEDA ST,488532822539,COMPLIED,90058,2021,011,"50,000 to 99,999 sqft",64.1,Not Available,Not Available,...,1,100,Not Available,Manufacturing/Industrial Plant,16.3,45.6,244.3,16.3,45.6,2007


In [10]:
results_clean_df = results_df.loc[:,['building_id', 'total_ghg_emissions', 'occupancy', 'primary_property_1', 'property_gfa_1', 'site_eui', 'source_eui',\
'water_use', 'weather_normalized_3', 'weather_normalized_4', 'year_built']]
results_clean_df = results_clean_df.dropna()
results_clean_df

,building_id,total_ghg_emissions,occupancy,primary_property_1,property_gfa_1,site_eui,source_eui,water_use,weather_normalized_3,weather_normalized_4,year_built
1,476102819647,138.1,100,Multifamily Housing,44769,54.7,99.9,2859.8,55.3,100.5,2001
2,427804836294,170.6,75,Office,84850,30.3,84.9,1052.3,Not Available,Not Available,1975
3,448460885789,65.9,100,Multifamily Housing,31030,38.4,59.8,1.6,39.1,60.6,2010
4,488532822539,64.1,100,Manufacturing/Industrial Plant,65000,16.3,45.6,244.3,16.3,45.6,2007
5,455587859077,33.5,100,Multifamily Housing,27777,20,40.3,837.1,20,40.3,1961
...,...,...,...,...,...,...,...,...,...,...,...
52578,487878840146,230.3,100,Multifamily Housing,105715,38.2,75.6,12530.6,37.8,75.1,1996
52579,481125799862,118,100,Multifamily Housing,69943,29.8,56.4,2476.1,29.1,55.7,2003
52580,379589902773,59.7,100,Multifamily Housing,28821,36.6,68.3,2283.1,36.6,68.3,1962
52581,410020916912,68.2,100,Multifamily Housing,21506,56.1,103.7,1468.4,57.4,105,1962


In [11]:
property_types_counts = results_clean_df.groupby(['primary_property_1'])['building_id'].count().sort_values(ascending = False)
property_types_counts

primary_property_1
Multifamily Housing                   14050
Office                                 3865
Manufacturing/Industrial Plant         1546
Non-Refrigerated Warehouse             1342
Distribution Center                     758
                                      ...  
Convenience Store with Gas Station        2
Stationnement                             1
Roller Rink                               1
Courthouse                                1
Bureau                                    1
Name: building_id, Length: 80, dtype: int64

In [12]:
keep_property_types = property_types_counts.loc[property_types_counts > 300]
keep_property_types = keep_property_types.drop(labels=['Other'])
keep_property_types = list(keep_property_types.index)
keep_property_types

['Multifamily Housing',
 'Office',
 'Manufacturing/Industrial Plant',
 'Non-Refrigerated Warehouse',
 'Distribution Center',
 'Self-Storage Facility',
 'Retail Store',
 'Parking',
 'Mixed Use Property',
 'Fire Station',
 'Other - Recreation',
 'Hotel',
 'Medical Office',
 'Supermarket/Grocery Store',
 'Strip Mall']

In [14]:
buildings_df = results_clean_df.loc[results_clean_df['primary_property_1'].isin(keep_property_types)]
buildings_df

,building_id,total_ghg_emissions,occupancy,primary_property_1,property_gfa_1,site_eui,source_eui,water_use,weather_normalized_3,weather_normalized_4,year_built
1,476102819647,138.1,100,Multifamily Housing,44769,54.7,99.9,2859.8,55.3,100.5,2001
2,427804836294,170.6,75,Office,84850,30.3,84.9,1052.3,Not Available,Not Available,1975
3,448460885789,65.9,100,Multifamily Housing,31030,38.4,59.8,1.6,39.1,60.6,2010
4,488532822539,64.1,100,Manufacturing/Industrial Plant,65000,16.3,45.6,244.3,16.3,45.6,2007
5,455587859077,33.5,100,Multifamily Housing,27777,20,40.3,837.1,20,40.3,1961
...,...,...,...,...,...,...,...,...,...,...,...
52578,487878840146,230.3,100,Multifamily Housing,105715,38.2,75.6,12530.6,37.8,75.1,1996
52579,481125799862,118,100,Multifamily Housing,69943,29.8,56.4,2476.1,29.1,55.7,2003
52580,379589902773,59.7,100,Multifamily Housing,28821,36.6,68.3,2283.1,36.6,68.3,1962
52581,410020916912,68.2,100,Multifamily Housing,21506,56.1,103.7,1468.4,57.4,105,1962
